In [ ]:
# %load_ext autoreload
# %autoreload 2
# %load_ext lab_black

In [25]:
import numpy as np
import pandas as pd
import torch
import sys

from sklearn.manifold import TSNE
import plotly.graph_objects as go

sys.path.append("../")

## Loading Model and Vocabulary

In [26]:
folder = "weights/cbow_MalaysiaKini_200k"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load(f"{folder}/model.pt", map_location=device)
vocab = torch.load(f"{folder}/vocab.pt")

## Getting Embeddings

In [27]:
# embedding from first model layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms
embeddings_norm.shape

(12199, 300)

# Visualization with t-SNE

In [5]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
embeddings_df_trans.index = vocab.get_itos()

# if token is a number
is_numeric = embeddings_df_trans.index.str.isnumeric()

KeyboardInterrupt: 

In [ ]:
color = np.where(is_numeric, "green", "black")
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=color),
    )
)
fig.write_html("../word2vec_visualization.html")

# Find Similar Words

In [28]:
def get_top_similar(word: str, topN: int = 10):
    word_id = vocab[word]
    if word_id == 0:
        print("Out of vocabulary word")
        return

    word_vec = embeddings_norm[word_id]
    word_vec = np.reshape(word_vec, (len(word_vec), 1))
    dists = np.matmul(embeddings_norm, word_vec).flatten()
    topN_ids = np.argsort(-dists)[1 : topN + 1]

    topN_dict = {}
    for sim_word_id in topN_ids:
        sim_word = vocab.lookup_token(sim_word_id)
        topN_dict[sim_word] = dists[sim_word_id]
    return topN_dict

In [29]:
vocab.get_itos()

['<unk>',
 ',',
 '.',
 'di',
 'yang',
 'yg',
 'dan',
 'dn',
 ')',
 '(',
 'dengan',
 'pada',
 'ini',
 'itu',
 'kepada',
 'untuk',
 'utk',
 'dalam',
 'mereka',
 'saya',
 'malaysia',
 'negara',
 'tak',
 'tidak',
 'lebih',
 'ni',
 'ke',
 'telah',
 'oleh',
 'berkata',
 'sebagai',
 'akan',
 'akn',
 'katanya',
 'atau',
 'ia',
 'lagi',
 'boleh',
 'beliau',
 'dlm',
 'dlam',
 'satu',
 "'",
 'kerana',
 'menteri',
 'menjadi',
 'kita',
 'parti',
 'selepas',
 'juga',
 'dari',
 'seperti',
 'tu',
 'tue',
 'tersebut',
 'gak',
 'secara',
 'negeri',
 'bagi',
 'ketika',
 'daripada',
 'drpd',
 'tahun',
 'perlu',
 'serta',
 'kami',
 'masa',
 'bahawa',
 'dapat',
 'kerajaan',
 'politik',
 'covid-19',
 'sebelum',
 'johor',
 'adalah',
 'lain',
 'semua',
 'adlh',
 'rakyat',
 'dua',
 'ada',
 'umno',
 'turut',
 'antara',
 'datuk',
 'thn',
 'pihak',
 'syarikat',
 'bukan',
 '-',
 'dr',
 'pilihan',
 'apabila',
 'kata',
 'de',
 'hanya',
 'masih',
 'dunia',
 'peratus',
 'kuala',
 'raya',
 'hari',
 'ahli',
 'ekonomi',
 

In [37]:
get_top_similar("make")

{'get': 0.28866112,
 'be': 0.27543044,
 'financial': 0.24694407,
 'mike': 0.24102193,
 'elon': 0.23901844,
 'last': 0.23324665,
 'service': 0.22750519,
 'can': 0.22327247,
 'moh': 0.22266234,
 'love': 0.2203166}

# Vector Equations

In [22]:
emb1 = embeddings[vocab["king"]]
emb2 = embeddings[vocab["man"]]
emb3 = embeddings[vocab["woman"]]

emb4 = emb1 - emb2 + emb3
emb4_norm = (emb4 ** 2).sum() ** (1 / 2)
emb4 = emb4 / emb4_norm

emb4 = np.reshape(emb4, (len(emb4), 1))
dists = np.matmul(embeddings_norm, emb4).flatten()

top5 = np.argsort(-dists)[:5]

for word_id in top5:
    print("{}: {:.3f}".format(vocab.lookup_token(word_id), dists[word_id]))

king: 0.554
woman: 0.462
æthelfrith: 0.334
jacobites: 0.300
copeland: 0.296
